## The Steady State Heat solver

-----------

2D, Steady Heat Equation with no variation in the x direction

\\[
0 = k \frac{\partial^2 T}{\partial y^2}+H
\\]

\\[
\frac{\partial T}{\partial x} = 0
\\]

with $0 \leqslant x \leqslant 1 $ and $ y_{0}\leqslant y \leqslant y_{1}$


Two models are presented below, each with an analytic solution that the numerical results are tested against.

The general solution is:

$ T = -\frac{h}{2 k}y^{2} + c_{0}y + c_{1} $

where $c_{0}, c_{1}$ are arbitrary constants found by applying each model's boundary conditions

In [ ]:
# # analytic solution definitions
def analyticTemperature(y, h, k, c0, c1):
     return -h/(2*k)*y*y + c0*y + c1

def exactDeriv(y, h, k, c0):
    return -h/k*y + c0

In [ ]:
import underworld as uw
import glucifer
import numpy as np
rank = uw.rank()

# for machines without matplotlib #
make_graphs = True
try:
    import matplotlib
except ImportError:
    make_graphs=False

# depth range
y0 = -.60
y1 = 1.3

# build mesh and fields
mesh = uw.mesh.FeMesh_Cartesian( elementType = ("Q1"), 
                                 elementRes  = (10, 20), 
                                 minCoord    = (0., y0), 
                                 maxCoord    = (1., y1))

tField    = uw.mesh.MeshVariable(mesh=mesh, nodeDofCount=1, dataType='double')
topWall = mesh.specialSets['MaxJ_VertexSet']
bottomWall = mesh.specialSets['MinJ_VertexSet']

## Model 1)


2D, Heat Equation with a Dirichlet BC (top) and a Neumann BC (bottom).

$T(x,y_{1}) = T_{1}$

$ -k\nabla{T} \mid _{y_{0}} = \left[\,0.0,\,f\,\right] $


------

Effectively a 1D problem in $y$-axis - arbitrary constants are:

$c_{0} = f + \frac{h}{k}y_{0}$

$c_{1} = T_{1} + \frac{h}{2 k}y_{1}^2 - c_{0}y_{1} $


In [ ]:
T1 = 8.0       # surface temperature
k = 0.70        # diffusivity
h = 9.0     # heat production, source term
f = -5.23 

# analytic solution definitions
# 1 dirichlet conditions (top) + 1 neumann (bottom)
c0 = 1.0/k*(h*y0-f)
c1 = T1 + h/(2.0*k)*y1**2 - c0*y1

In [ ]:
for ii in topWall:
    tField.data[ii] = T1

# define neumann condition
nbc = uw.conditions.NeumannCondition( flux=[0.0,f], 
                                      variable=tField, 
                                      nodeIndexSet=(bottomWall) )

# flag the dirichlet conditions on the topWall only
bc = uw.conditions.DirichletCondition(tField, indexSetsPerDof=(topWall) )

# define heat eq. system
ss = uw.systems.SteadyStateHeat( temperatureField  = tField,
                                 fn_diffusivity = k,
                                 fn_heating     = h,
                                 conditions     = [bc, nbc] ) 

In [ ]:
solver = uw.systems.Solver(ss)
solver.solve()

In [ ]:
# create numpy arrays for analytics
yvals       = np.zeros(len(mesh.specialSets['MinI_VertexSet'].data))
ycoord      = np.zeros_like(yvals)
analytic    = np.zeros_like(yvals)

ids = mesh.specialSets['MinI_VertexSet']
yvals[:] = tField.evaluate(ids).T

ycoord = tField.mesh.data[ids.data,[1]]
analytic = analyticTemperature(ycoord, h, k, c0, c1)

abserr = uw.utils._nps_2norm(analytic - yvals)
mag = uw.utils._nps_2norm(analytic)
relerr = abserr / mag

from mpi4py import MPI
comm = MPI.COMM_WORLD
# assuming order in the allgather is the same
coords = comm.allgather(ycoord)
numerical = comm.allgather(yvals)

if rank == 0:
    # build matplot lib graph of result only on proc 0

    # 1st build exact solution hiRes
    big = np.linspace(y0,y1)
    cool = analyticTemperature(big, h, k, c0, c1)

    %matplotlib inline
    import matplotlib.pyplot as pyplot
    import matplotlib.pylab as pylab
    pylab.rcParams[ 'figure.figsize'] = 12, 6
    pyplot.plot(coords, numerical, 'o', color = 'black', label='numerical') 
    pyplot.plot(big, cool, color = 'red', label="exact") 
    pyplot.xlabel('Y coords')
    pyplot.ylabel('Temperature')
    pyplot.show()


if rank == 0:
    threshold = 3.0e-4
    yspot = y0
    print "Numerical flux at y = " ,yspot,"is", tField.fn_gradient.evaluate([0.2,yspot])[0][1]
    print "Exact     flux at y = " ,yspot,"is", exactDeriv(yspot,h,k,c0)
    print "|T - T_analytic|_2 = {0:.3e}\n\n|T|_2 = {1:.3e}\n".format(abserr, mag)
    print "Rel. error = {0:.3e}".format(relerr)
    if relerr > threshold:
        raise RuntimeError("The numerical solution is outside the error threshold of the analytic solution." \
                           "The Relative error was ", relerr," the threshold is ", threshold)

## Model 2) 

2D, Steady State Heat Equation with Dirichlet BC at top and bottom boundary.

$T(x,y_{1}) = T_{1}$

$ T(x,y_{0}) = T_{0} $

------

arbitrary constants are:

$ c_{0} = \frac{1}{y_{1}-y_{0}} \left[ T_{1}-T_{0}+\frac{h} {2k}(y_{1}^2-y_{0}^2) \right] $

$c_{1} = T_{1} + \frac{h}{2k}y_{1}^2 - c_{0}y_{1}$


In [ ]:
# Model parameters
T1 = 8.0   # top surface temperature
T0 = 4.0   # bottom surface temperature
k = 0.50    # diffusivity
h = 1.0e0  # heat production, source term

# arbitrary constant given the 2 dirichlet conditions
c0 = (T1-T0+h/(2*k)*(y1**2-y0**2)) / (y1-y0)
c1 = T1 + h/(2*k)*y1**2 - c0*y1

In [ ]:
# set boundary conditions
for ii in topWall:
    tField.data[ii] = T1
for ii in bottomWall:
    tField.data[ii] = T0

# flag boundary conditions
bc = uw.conditions.DirichletCondition(tField, indexSetsPerDof=(topWall+bottomWall) )

# define heat eq. system
ss = uw.systems.SteadyStateHeat( temperatureField  = tField,
                                 fn_diffusivity = k,
                                 fn_heating     = h,
                                 conditions     = [bc] )

In [ ]:
solver = uw.systems.Solver(ss)
solver.solve()

In [ ]:
# create numpy arrays for analytics
yvals       = np.zeros(len(mesh.specialSets['MinI_VertexSet'].data))
ycoord      = np.zeros_like(yvals)
analytic    = np.zeros_like(yvals)

ids = mesh.specialSets['MinI_VertexSet']
yvals[:] = tField.evaluate(ids).T

ycoord   = tField.mesh.data[ids.data,[1]]
analytic = analyticTemperature(ycoord, h, k, c0, c1)

abserr = uw.utils._nps_2norm(analytic - yvals)
mag = uw.utils._nps_2norm(analytic)
relerr = abserr / mag

from mpi4py import MPI
comm = MPI.COMM_WORLD
# assuming order in the allgather is the same
coords = comm.allgather(ycoord)
numerical = comm.allgather(yvals)

In [ ]:
if rank == 0:
    threshold = 3.0e-4
    yspot = y0
    print "Numerical flux at y = " ,yspot,"is", tField.fn_gradient.evaluate([0.2,yspot])[0][1]
    print "Exact flux at y=" ,yspot,"is", exactDeriv(yspot,h,k,c0)
    print "\n|T - T_analytic|_2 = {0:.3e}\n\n|T|_2 = {1:.3e}\n".format(abserr, mag)
    print "Rel. error = {0:.3e}".format(relerr)
    if relerr > threshold:
        raise RuntimeError("The numerical solution is outside the error threshold of the analytic solution." \
                           "The Relative error was ", relerr," the threshold is ", threshold)  
        

In [ ]:

if rank == 0:
    # build matplot lib graph of result only on proc 0

    # 1st build exact solution hiRes
    big = np.linspace(y0,y1)
    cool = analyticTemperature(big, h, k, c0, c1)

    %matplotlib inline
    import matplotlib.pyplot as pyplot
    import matplotlib.pylab as pylab
    pylab.rcParams[ 'figure.figsize'] = 12, 6
    pyplot.plot(coords, numerical, 'o', color = 'black', label='numerical') 
    pyplot.plot(big, cool, color = 'red', label="exact") 
    pyplot.xlabel('Y coords')
    pyplot.ylabel('Temperature')
    pyplot.show()
